In [1]:
import pandas as pd
import numpy as np  # ⚠️ Sửa: Thêm thư viện numpy để xử lý NaN
from sklearn.preprocessing import StandardScaler

In [3]:
# --- 1. Đọc dữ liệu ---
file_path = "Data 30.xlsx"  # đổi tên nếu khác
df = pd.read_excel(file_path, sheet_name="FINAL")


In [4]:
# --- 2. Lựa chọn cột và đổi tên cột bị lỗi ---
# ⚠️ Sửa: Cột 'Source.Name' trong file bị lỗi 'Source.NaE6e', ta sẽ chọn và đổi tên
df = df.rename(columns={'Source.NaE6e': 'Source.Name'})

In [5]:
# Chọn các cột cần thiết
df = df[['Source.Name', 'Date', 'Price', 'Open', 'High', 'Low', 'Vol.', 'Change %']]

# --- 3. Làm sạch dữ liệu ---
df['Date'] = pd.to_datetime(df['Date'], errors='coerce')

In [7]:


# ⚠️ Sửa: Cột 'Change %' đã là số, không cần .str.replace('%')
df['Change %'] = pd.to_numeric(df['Change %'], errors='coerce')

In [8]:
# ⚠️ Sửa: Đảm bảo các cột giá khác cũng là số (an toàn hơn)
df['Price'] = pd.to_numeric(df['Price'], errors='coerce')
df['Open'] = pd.to_numeric(df['Open'], errors='coerce')
df['High'] = pd.to_numeric(df['High'], errors='coerce')
df['Low'] = pd.to_numeric(df['Low'], errors='coerce')

In [9]:
# Xóa các dòng có giá trị rỗng (sau khi chuyển đổi)
df = df.dropna(subset=['Date', 'Price', 'Open', 'High', 'Low', 'Vol.', 'Change %'])
df = df.drop_duplicates()

In [10]:
# --- 4. Tạo biến đặc trưng mới ---
# (Phần này giữ nguyên, giờ sẽ hoạt động vì dữ liệu đã sạch)
df['Daily Range'] = df['High'] - df['Low']
df = df.sort_values(by=['Source.Name', 'Date'])
df['Next_Price'] = df.groupby('Source.Name')['Price'].shift(-1)
df['Target'] = (df['Next_Price'] > df['Price']).astype(int)  # 1 nếu giá tăng, 0 nếu giảm

In [11]:
# --- 5. Tổng hợp dữ liệu trung bình theo mã cổ phiếu ---
# (Phần này giữ nguyên, giờ sẽ hoạt động)
df_mean = df.groupby('Source.Name').agg({
    'Price':'mean',
    'Vol.':'mean',
    'Change %':'mean',
    'Daily Range':'mean'
}).reset_index()

In [12]:
# --- 6. Chuẩn hóa dữ liệu số ---
# (Phần này giữ nguyên)
scaler = StandardScaler()
scaled_values = scaler.fit_transform(df_mean[['Price','Vol.','Change %','Daily Range']])
df_scaled = pd.DataFrame(scaled_values, columns=['Price_scaled','Vol_scaled','Change_scaled','Range_scaled'])
df_preprocessed = pd.concat([df_mean['Source.Name'], df_scaled], axis=1)

In [14]:
# --- 7. Xuất file kết quả ---
output_path = r"D:\Python\FINAL\VN30_preprocessed.xlsx"
df_preprocessed.to_excel(output_path, index=False)

print(f"✅ File tiền xử lý đã lưu tại: {output_path}")

✅ File tiền xử lý đã lưu tại: D:\Python\FINAL\VN30_preprocessed.xlsx
